In [2]:
from tqdm import tqdm

def import_labelled_data(folder_path="data/labelled", group_relevant=True, dev_sampling=True):
    import os
    import json

    xFeatures = []
    yLabels = []

    for subfolder in os.listdir(folder_path):
        # filter out top-level files
        if not os.path.isdir(os.path.join(folder_path, subfolder).replace("\\", "/")):
            continue

        sample_counter = -1

        for file in tqdm(
            os.listdir(os.path.join(folder_path, subfolder).replace("\\", "/"))
        ):
            sample_counter+=1
            # each file is a json file representing a single article
            # with fields like 'url', 'title', 'text', 'class', etc
            if file.endswith(".json") and (not dev_sampling or sample_counter % 100 == 0):
                raw = json.load(
                    open(
                        os.path.join(folder_path, subfolder, file).replace("\\", "/"),
                        encoding="latin-1",
                    )
                )
                raw["source_type"] = subfolder

                if group_relevant:
                    if raw["class"] == "irrelevant":
                        yLabels.append("irrelevant")
                    else:
                        yLabels.append("relevant")
                    xFeatures.append(raw)

                else:
                    yLabels.append(raw["class"])
                    xFeatures.append(raw)

    return xFeatures, yLabels


xFeatures, yLabels = import_labelled_data('../../data/labelled2')

100%|██████████| 10105/10105 [00:00<00:00, 400470.94it/s]


In [12]:
import pandas as pd
# include ylabels
df = pd.DataFrame(xFeatures)
df['class'] = yLabels
df = df[]
df.head()

,reference type,author,year,title,series editor,series/book title,place published,institution,publisher,date,doi,class,abstract,url,text,source_type,report number,folder
0,CE Study,None,None,Invasive Freshwater Species,None,None,None,Conservation Evidence Team,Conservation Evidence,None,None,relevant,None,None,\n \n \n Control of freshwater \n invasi...,synopses,NaN,NaN
1,Report,D. J. GLAVES; M. MORECROFT; C. FITZGIBBON; P. ...,2013,The effects of managed burning on upland peatl...,None,Natural England Evidence Review,None,Natural England,None,None,None,relevant,None,http://publications.naturalengland.org.uk/publ...,We use some essential cookies to make this ser...,scraped,Report No. NEER004,4. Invertebrates
2,CE Study,"Pearse A.T., Cavitt J.F. & Cully J.F. (2004) E...",2004,Effects of food supplementation on female nest...,None,None,None,Conservation Evidence Team,Conservation Evidence,None,None,relevant,None,https://www.conservationevidence.com/individua...,"A randomised, replicated and controlled trial ...",studies,3765,NaN
3,CE Study,"Feehan J., Gillmor D. & Culleton N. (2005) Eff...",2005,Effects of an agri-environment scheme on farml...,None,None,None,Conservation Evidence Team,Conservation Evidence,None,None,relevant,None,https://www.conservationevidence.com/individua...,A replicated paired sites comparison study in ...,studies,2645,NaN
4,CE Study,"Arellano M.L., Velasco M.A., Martínez Aguirre ...",2018,Management strategy to avoid chytrid fungus in...,None,None,None,Conservation Evidence Team,Conservation Evidence,None,None,relevant,None,https://www.conservationevidence.com/individua...,Eggs which were removed from a chytrid-infecte...,studies,6856,NaN


In [6]:
label_to_int = {label: idx for idx, label in enumerate(set(yLabels))}
int_to_label = {idx: label for label, idx in label_to_int.items()}

In [8]:
from datasets import Dataset
from chunking import chunk_datapoint

dataset = Dataset.from_list([{'feature':xFeature,'label':yLabel} for xFeature, yLabel in zip(xFeatures, map(label_to_int.get,yLabels))])
dataset.train_test_split(test_size=0.2)

dataset.map(lambda record: {'feature':chunk_datapoint(record['feature'],record['label'],chunk_size=8192,overlap=8192*0.2)},batched=True,num_proc=8)

ArrowInvalid: Could not convert 'Report No. NEER004' with type str: tried to convert to int64